<a href="https://colab.research.google.com/github/AntonLissa/trajectory_prediction/blob/main/get_trajectory_data_from_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get cars trajectories using supervision

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd 'drive/MyDrive/CV_project'

!pip install supervision

from IPython import display
display.clear_output()
import supervision as sv

!pip install ultralytics

import ultralytics
display.clear_output()


import numpy as np
from ultralytics import YOLO

In [2]:

model = YOLO('yolov8x.pt')
model.fuse

CLASS_NAMES_DICT = model.model.names
selected_classes = [2,3,5,7] # cars, bus, motorbikes, trucks

In [5]:
# settings
file_name = 'incrocio_1h'
SOURCE_VIDEO_PATH = 'videos/' + file_name + '.mp4'
TARGET_VIDEO_PATH = 'videos/processed/' + file_name +'_processed.mp4'
info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
frames_to_process = 2 # we process one frame every 2 for faster execution
print(info)

VideoInfo(width=1280, height=680, fps=30, total_frames=102449)


In [8]:

from collections import defaultdict

# Creare un dizionario per salvare gli ID degli oggetti e i rispettivi centri
object_centers = defaultdict(list)

byte_tracker = sv.ByteTrack(track_thresh=0.5, track_buffer=30, match_thresh=0.8, frame_rate=30)


# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    if index % frames_to_process == 0: # for fastere execution use 1 frame every frame_to_process
        print(index, '/', info.total_frames)

        # model prediction on single frame and conversion to supervision Detections
        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        # only consider class id from selected_classes define above
        detections = detections[np.isin(detections.class_id, selected_classes)]
        # tracking detections
        detections = byte_tracker.update_with_detections(detections)

        # add object centers to the id
        for box, id in zip(detections.xyxy, detections.tracker_id):
                center = (
            int((box[0] + box[2]) / 2),  # center X coord
            int((box[1] + box[3]) / 2)   # center Y coord
        )
                object_centers[id].append(center)

        return  frame

# process the whole video
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)

SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.


0 / 102449
200 / 102449
400 / 102449
600 / 102449
800 / 102449
1000 / 102449
1200 / 102449
1400 / 102449
1600 / 102449
1800 / 102449
2000 / 102449
2200 / 102449
2400 / 102449
2600 / 102449
2800 / 102449
3000 / 102449
3200 / 102449
3400 / 102449
3600 / 102449
3800 / 102449
4000 / 102449
4200 / 102449
4400 / 102449
4600 / 102449
4800 / 102449
5000 / 102449
5200 / 102449
5400 / 102449
5600 / 102449
5800 / 102449
6000 / 102449
6200 / 102449
6400 / 102449
6600 / 102449
6800 / 102449
7000 / 102449
7200 / 102449
7400 / 102449
7600 / 102449
7800 / 102449
8000 / 102449
8200 / 102449
8400 / 102449
8600 / 102449
8800 / 102449
9000 / 102449
9200 / 102449
9400 / 102449
9600 / 102449
9800 / 102449
10000 / 102449
10200 / 102449
10400 / 102449
10600 / 102449
10800 / 102449
11000 / 102449
11200 / 102449
11400 / 102449
11600 / 102449
11800 / 102449
12000 / 102449
12200 / 102449
12400 / 102449
12600 / 102449
12800 / 102449
13000 / 102449
13200 / 102449
13400 / 102449
13600 / 102449
13800 / 102449
14000 /

## Check the data obtained from the detections

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import numpy as np

# Carica il frame dal video
cap = cv2.VideoCapture('videos/incrocio_mod.mp4')
ret, frame = cap.read()
cap.release()

plt.figure(figsize=(16, 10))
# Visualize a frame from the video
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
print('numero di traiettorie:', len(object_centers.items()))
# Draw trajectories
for key, value in object_centers.items():
    x, y = zip(*value)
    plt.scatter(x, y, s = 3, c='blue')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Trajectories')
plt.grid(True)

plt.show()

# Save data into a csv file

In [ ]:
import pandas as pd
csv_file_path = 'trajectories/' + file_name +'.csv'


data = []
for key, value in object_centers.items():
    trajectory = {
        'Key': key,
        'X': [point[0] for point in value],
        'Y': [point[1] for point in value]
    }
    data.append(trajectory)

df = pd.DataFrame(data)

df.to_csv(csv_file_path, index=False)